<a href="https://colab.research.google.com/github/HairuoGao029/MasterThesis/blob/main/dataset_attacker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from xml.sax import parse, handler
import matplotlib.pyplot as plt
import os, random, shutil
import ast
import math

from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive

Mounted at /content/drive
/content/drive/My Drive


In [ ]:
# d20 as test set
# Fake and Extension
# 30%-5, 50%-10, 70%-12

class DetectorParse(handler.ContentHandler):
    """reading in the dataset from xml file
    """
    def __init__(self):
        self.lanes = {}
        self.sample = {}
        self.lane_list = ['56250003#0_0%1','237111029#1.254_0%0','237111029#1.254_0%1','237111029#1.254_1%0','237111029#1.254_1%1','238459506.6_0%1','62830645#2.0.0_0%0','62830645#2.0.0_0%1','62830645#2.0.0_1%0','62830645#2.0.0_1%1']


    def startElement(self, name, attrs):
        if name == 'timestep':
            self.probe_id = attrs['id']
            self.time = float(attrs['time'])
            
        if name == 'vehicle':  
            pos = float(attrs['pos'])
            pos_i = str(int(pos//1000))
            lane_id = attrs['lane']+'%'+pos_i
            if lane_id in self.lane_list and lane_id not in self.lanes:
                self.lanes[lane_id] = {}
                self.lanes[lane_id]['Speed'] = []
                self.lanes[lane_id]['ID'] = []
                #self.lanes[lane_id]['AverageSpeed'] = []
                self.sample[lane_id] = {}
                self.sample[lane_id]['Original'] = []
                self.sample[lane_id]['Attacker'] = []
            if (lane_id in ['56250003#0_0%1','237111029#1.254_0%0','237111029#1.254_0%1','237111029#1.254_1%0','237111029#1.254_1%1'] and self.time >= 10800.00 and self.time <= 13200.00) or (lane_id in ['238459506.6_0%1','62830645#2.0.0_0%0','62830645#2.0.0_0%1','62830645#2.0.0_1%0','62830645#2.0.0_1%1'] and self.time >= 20400.00 and self.time <= 21600.00):
        
                self.lanes[lane_id]['Speed'].append(float(attrs['speed']))
                self.lanes[lane_id]['ID'].append(str(attrs['id']))

        
  
    def endElement(self, name):
        if name == 'timestep':
            if self.probe_id == 'probe2':
              for key, value in self.lanes.items():
                if (key in ['56250003#0_0%1','237111029#1.254_0%0','237111029#1.254_0%1','237111029#1.254_1%0','237111029#1.254_1%1'] and self.time >= 10800.00 and self.time <= 13200.00) or (key in ['238459506.6_0%1','62830645#2.0.0_0%0','62830645#2.0.0_0%1','62830645#2.0.0_1%0','62830645#2.0.0_1%1'] and self.time >= 20400.00 and self.time <= 21600.00):  
                #if (key in ['237111029#1.254_0%0','237111029#1.254_0%1'] and self.time >= 10800.00 and self.time <= 13200.00):  
                
                #15:00-15:40 #17:40-18:00
                  n = len(value['Speed'])
                  if n == 1:
                      #rate = 0.3  # sample of benign users
                      #picknumber = math.ceil(n * rate)
                      #sample = random.sample(list(zip(value['Speed'], value['ID'])), picknumber)
                      self.sample[key]['Original'].append(list(zip(value['Speed'], value['ID'])))
                      self.sample[key]['Attacker'].append('None')
                      
                      #ave_speed = np.mean(speed)

                  elif n > 1:
                      rate = 0.3  # sample of benign users
                      picknumber = math.ceil(n * rate)
                      sample = random.sample(list(zip(value['Speed'], value['ID'])), picknumber)
                      #self.sample[key]['Original'].append(sample)
                      speed, id = zip(*sample)
                      std = np.std(speed)

                      rate_att = 0.3  # sample of attackers
                      picknumber_att = math.ceil(n * rate_att)
                      value['ID'] += 'att'
                      remain_index = tuple(set(range(len(list(zip(value['Speed'], value['ID']))))) - set(sample))
                      att_index = random.sample(remain_index, picknumber_att)
                      sample_att = [list(zip(value['Speed'], value['ID']))[i] for i in att_index]
                      #self.sample[key]['Original'].append(sample_att)

                      speed_att, id_att = zip(*sample_att)
                      speed_att_new = np.random.normal(5, 0.5, len(speed_att))
                      '''
                      ave_speed_tmp = []
                      ave_speed_tmp.append(speed)
                      ave_speed_tmp.append(speed_att_new)
                      ave_speed = np.mean(ave_speed_tmp)
                      '''
                      sample += list(zip(speed_att_new, id_att))
                      self.sample[key]['Original'].append(sample)
                      self.sample[key]['Attacker'].append(id_att)
                      
                  #self.lanes[key]['AverageSpeed'].append(ave_speed)
                  self.lanes[key]['Speed'] = []
                  self.lanes[key]['ID'] = []
                           
    
    def endDocument(self):
        #print(self.lanes)
        print(self.sample)

In [ ]:
# d20 as test set
# Mask
# 30%-5, 50%-10, 70%-12

class DetectorParse(handler.ContentHandler):
    """reading in the dataset from xml file
    """
    def __init__(self):
        self.lanes = {}
        self.sample = {}
        self.lane_list = ['238459506.6_0%1','62830645#2.0.0_0%0','62830645#2.0.0_0%1','62830645#2.0.0_1%0','62830645#2.0.0_1%1']


    def startElement(self, name, attrs):
        if name == 'timestep':
            self.probe_id = attrs['id']
            self.time = float(attrs['time'])
            
        if name == 'vehicle':  
            pos = float(attrs['pos'])
            pos_i = str(int(pos//1000))
            lane_id = attrs['lane']+'%'+pos_i
            if lane_id in self.lane_list and lane_id not in self.lanes:
                self.lanes[lane_id] = {}
                self.lanes[lane_id]['Speed'] = []
                self.lanes[lane_id]['ID'] = []
                #self.lanes[lane_id]['AverageSpeed'] = []
                self.sample[lane_id] = {}
                self.sample[lane_id]['Original'] = []
                self.sample[lane_id]['Attacker'] = []
            if lane_id in ['238459506.6_0%1','62830645#2.0.0_0%0','62830645#2.0.0_0%1','62830645#2.0.0_1%0','62830645#2.0.0_1%1'] and self.time >= 19200.00 and self.time <= 20400.00:
        
                self.lanes[lane_id]['Speed'].append(float(attrs['speed']))
                self.lanes[lane_id]['ID'].append(str(attrs['id']))

        
  
    def endElement(self, name):
        if name == 'timestep':
            if self.probe_id == 'probe2':
              for key, value in self.lanes.items():
                if key in ['238459506.6_0%1','62830645#2.0.0_0%0','62830645#2.0.0_0%1','62830645#2.0.0_1%0','62830645#2.0.0_1%1'] and self.time >= 19200.00 and self.time <= 20400.00:
                #17:00-17:40 #17:00-17:20
                  n = len(value['Speed'])
                  if n == 1:
                      #rate = 0.3  # sample of benign users
                      #picknumber = math.ceil(n * rate)
                      #sample = random.sample(list(zip(value['Speed'], value['ID'])), picknumber)
                      self.sample[key]['Original'].append(list(zip(value['Speed'], value['ID'])))
                      self.sample[key]['Attacker'].append('None')
                      
                      #ave_speed = np.mean(speed)

                  elif n > 1:
                      rate = 0.3  # sample of benign users
                      picknumber = math.ceil(n * rate)
                      sample = random.sample(list(zip(value['Speed'], value['ID'])), picknumber)
                      #self.sample[key]['Original'].append(sample)
                      speed, id = zip(*sample)
                      std = np.std(speed)

                      rate_att = 0.7  # sample of attackers
                      picknumber_att = math.ceil(n * rate_att)
                      remain_index = tuple(set(range(len(list(zip(value['Speed'], value['ID']))))) - set(sample))
                      att_index = random.sample(remain_index, picknumber_att)
                      sample_att = [list(zip(value['Speed'], value['ID']))[i] for i in att_index]
                      #self.sample[key]['Original'].append(sample_att)

                      speed_att, id_att = zip(*sample_att)
                      if key == '238459506.6_0%1':
                        speed_att_new = np.random.normal(25, 0.5, len(speed_att))
                      else:
                        speed_att_new = np.random.normal(32, 0.5, len(speed_att))
                      '''
                      ave_speed_tmp = []
                      ave_speed_tmp.append(speed)
                      ave_speed_tmp.append(speed_att_new)
                      ave_speed = np.mean(ave_speed_tmp)
                      '''
                      sample += list(zip(speed_att_new, id_att))
                      self.sample[key]['Original'].append(sample)
                      self.sample[key]['Attacker'].append(id_att)
                      
                  #self.lanes[key]['AverageSpeed'].append(ave_speed)
                  self.lanes[key]['Speed'] = []
                  self.lanes[key]['ID'] = []
                           
    
    def endDocument(self):
        #print(self.lanes)
        print(self.sample)

In [ ]:
detector_output = DetectorParse()
parse('/content/drive/MyDrive/MSc/datasets/rawdata/d20_12-24.xml', detector_output) # reading in the dataset from xml file
# segment_id = lane_id % number of the kilometer segment (e.g. 23024319#1.397_1%0)
# 632 road segments in total

# Save
#np.save('/content/drive/MyDrive/MSc/datasets/0913/individual_FE_30.npy', detector_output.sample) # data storage as a dict: out_day_time[segment_id]['AverageSpeed']

{'62830645#2.0.0_0%0': {'Original': [], 'Attacker': []}, '237111029#1.254_0%0': {'Original': [[(30.12, '32427-CLONEC1-1-1-i1-CLONEC1')], [(35.54, '31248-CLONEC1-1-i1-CLONEC1'), (4.648743510045836, '32427-CLONEC1-1-1-i1-CLONEC1')], [(35.73, '31248-CLONEC1-1-i1-CLONEC1'), (4.757356310445383, '31248-CLONEC1-1-i1-CLONEC1')], [(35.7, '31248-CLONEC1-1-i1-CLONEC1'), (4.981250900545856, '35289-CLONEC1-0-CLONEC1-0-CLONEC1-i1-CLONEC1')], [(40.26, '35289-CLONEC1-0-CLONEC1-0-CLONEC1-i1-CLONEC1'), (5.070070335189813, '30055-CLONEC1-1-i1-CLONEC1')], [(24.59, '32427-CLONEC1-0-1-CLONEC1-i1'), (4.829796458587702, '30055-CLONEC1-1-i1-CLONEC1')], [(30.85, '26416-CLONEC1-0-i2-CLONEC1'), (32.81, '35289-CLONEC1-0-CLONEC1-0-i1'), (4.984461149937435, '32427-CLONEC1-0-1-CLONEC1-i1'), (5.523343017821661, '35289-CLONEC1-0-CLONEC1-0-i1')], [(34.01, '35289-CLONEC1-1-0-CLONEC1-i1-CLONEC1'), (24.72, '32427-CLONEC1-0-1-CLONEC1-i1'), (32.1, '35289-CLONEC1-0-CLONEC1-0-CLONEC1-i1'), (4.990678495644528, '34980-i1'), (4.6